In this notebook we will see how to create safetensors 

We have to keep the following things in mind 
- [ ] one csv and one pt files
- [ ] can have multiple csv files and multiple pt files
- [ ] preprocessing before saving as safe tensor
- [ ] we will save only bboxes once. - there is an idea to store multiple different bounding boxes for the same image but we have to handle this at all transforms. keeping this as TODO for now 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#| default_exp create_ds

In [3]:
#| export 
import torch
import pandas as pd
import numpy as np 
import fastcore.all as fc 

from loguru import logger
from typing import Optional
from safetensors.numpy import save_file
from qct_utils.ctscan.ct_loader import CTAnnotLoader

/home/users/shubham.kumar/miniconda3/envs/qct_deep_clone/lib/python3.9/site-packages/scipy-1.9.1-py3.9-linux-x86_64.egg/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from qct_data.series_meta import get_qct_collection

In [5]:
from qct_utils.ctvis.viewer import plot_scans
from qct_utils.ctvis.vis_cct import plot_boxes_ctscan
from qct_utils.ctvis.draw import draw_bboxes_and_titles

In [6]:
datasets = ['dedomena',
 'dsb',
 'internal',
 'lidc',
 'medframe',
 'mgh',
 'nlst',
 'qxr_fda',
 'segmed_pm',
 'recist_segmed',
 'wcg']

In [7]:
dataset = "recist_segmed"

In [8]:
# Specify the directory path you want to create
save_dir_root = fc.Path(f"/cache/fast_data_nas8/qct/shubham/cache_3d/{dataset}/")
save_dir_root.mkdir(parents=True, exist_ok=True)

lungmask_path = fc.Path(f"/cache/fast_data_nas8/qct/shubham/lung_mask_cache_det/{dataset}/")
lungmask_path.mkdir(parents=True, exist_ok=True)

In [10]:
csv_loc = f"/home/users/shubham.kumar/projects/qct_data/qct_data_status/{dataset}/data/merged_new.csv"
df = pd.read_csv(csv_loc)
df.head()

,scan_name,annot_union,meta_union,rads
0,1.3.6.1.4.1.55648.2.82945951966077009162017767...,"{'mask': {'rle': {'encoding': (176, 262, 33, 8...",{'recist_lesion_type': 'NTL|NTL'},2
1,1.3.6.1.4.1.55648.2.82945951966077009162017767...,"{'mask': {'rle': {'encoding': (197, 338, 16, 2...",{'recist_lesion_type': 'TL1|TL2'},2
2,1.3.6.1.4.1.55648.2.82945951966077009162017767...,"{'mask': {'rle': {'encoding': (203, 370, 26, 5...",{'recist_lesion_type': 'TL2|NTL'},2
3,1.3.6.1.4.1.55648.2.82945951966077009162017767...,"{'mask': {'rle': {'encoding': (119, 215, 35, 6...",{'recist_lesion_type': 'TL3|TL1'},2
4,1.3.6.1.4.1.55648.2.82945951966077009162017767...,"{'mask': {'rle': {'encoding': (348, 265, 25, 3...",{'recist_lesion_type': 'NTL|TL3'},2


In [11]:
df.shape

(1816, 4)

### Making annot df compatible for CTAnnotLoader

In [11]:
df = df.rename(columns = {"annot_union" : "annot" , "meta_union": "meta"})
df = df.sort_values(by='scan_name')
df['counter'] = df.groupby('scan_name').cumcount() + 1
df['annot_id'] = df['scan_name'] + "_" + df['counter'].astype(str)
df.drop(columns=['counter'], inplace=True)
df.head()

,scan_name,annot,meta,rads,annot_id
975,1.3.6.1.4.1.55648.2.10090658896692464329125180...,"{'mask': {'rle': {'encoding': (302, 294, 61, 3...",{},1,1.3.6.1.4.1.55648.2.10090658896692464329125180...
976,1.3.6.1.4.1.55648.2.10090658896692464329125180...,"{'mask': {'rle': {'encoding': (218, 348, 55, 2...",{},1,1.3.6.1.4.1.55648.2.10090658896692464329125180...
723,1.3.6.1.4.1.55648.2.10097783833891709104772397...,"{'mask': {'rle': {'encoding': (179, 270, 24, 1...",{},2,1.3.6.1.4.1.55648.2.10097783833891709104772397...
722,1.3.6.1.4.1.55648.2.10097783833891709104772397...,"{'mask': {'rle': {'encoding': (321, 302, 69, 1...",{},2,1.3.6.1.4.1.55648.2.10097783833891709104772397...
141,1.3.6.1.4.1.55648.2.10124173454758677545232283...,"{'mask': {'rle': {'encoding': (336, 312, 54, 3...",{},1,1.3.6.1.4.1.55648.2.10124173454758677545232283...


## we need to see what all files we need to create the safetensors for?

In [12]:
series_ids = fc.L(df.scan_name.unique().tolist())
series_ids

(#589) ['1.3.6.1.4.1.55648.2.100906588966924643291251806135982148156','1.3.6.1.4.1.55648.2.10097783833891709104772397994918883764','1.3.6.1.4.1.55648.2.101241734547586775452322831296468201972','1.3.6.1.4.1.55648.2.106046966172856908668694384404129205184','1.3.6.1.4.1.55648.2.108093775649662319973295460215823794828','1.3.6.1.4.1.55648.2.108780287787214039585718901661926337254','1.3.6.1.4.1.55648.2.108994864792388198169781035980676401557','1.3.6.1.4.1.55648.2.110105676381256847997614363345304480315','1.3.6.1.4.1.55648.2.110564522454365172417303756477463130619','1.3.6.1.4.1.55648.2.112114183081112621537983224655019481596'...]

In [13]:
from qct_data.series_meta import get_qct_collection

In [14]:
collections = {"wcg": "wcg_fda_qct", "internal": "qure_internal", "qxr_fda": "wcg_fda_qxr","recist":"recist_segmed"}
for d in datasets :
    if d not in collections.keys() : 
        collections.update({d : d})

In [15]:
collections

{'wcg': 'wcg_fda_qct',
 'internal': 'qure_internal',
 'qxr_fda': 'wcg_fda_qxr',
 'recist': 'recist_segmed',
 'dedomena': 'dedomena',
 'dsb': 'dsb',
 'lidc': 'lidc',
 'medframe': 'medframe',
 'mgh': 'mgh',
 'nlst': 'nlst',
 'segmed_pm': 'segmed_pm',
 'recist_segmed': 'recist_segmed'}

In [16]:
meta_root = get_qct_collection(collection_name= collections[dataset])

In [17]:
ds = CTAnnotLoader(scans_root=meta_root, csv_loc=df, series_ids=series_ids)

In [18]:
sid = series_ids[np.random.randint(len(series_ids))]
ctscan = ds[sid]

In [19]:
for annot in ctscan.nodules:
    print(annot.gt.annot_id)
    print(annot.gt.annot.bbox)

1.3.6.1.4.1.55648.2.54139456488540773870566095168236824695_1
xc=367.5 yc=288.0 w=102.0 h=81.0 zc=73.5 d=52.0


In [20]:
kk = plot_boxes_ctscan(ctscan)

In [21]:
plot_scans([kk, ], ["scan"])

interactive(children=(RadioButtons(description='Slice plane selection:', index=2, options=('y-z', 'z-x', 'x-y'…

> Everything seems okay. lets process this and stip of the extra un-necessary stuff

In [22]:
np.unique(ctscan.scan.array)

array([-3024, -1090, -1088, ...,  3081,  3082,  3083], dtype=int32)

In [23]:
file = {}
spacing = ctscan.scan.spacing
file["series_id"] = ctscan.series_instance_uid
file["images"] = ctscan.scan.array.astype(np.int16)
file["boxes"] =  np.empty((0, 6)) if len(ctscan.nodules) == 0 else np.asarray([i.gt.annot.bbox.xcyczcwhd for i in  ctscan.nodules])[:, [1, 0, 2, 4, 3, 5]]
if np.any(file["boxes"]<0): breakpoint()
file["labels"] = np.zeros(file["boxes"].shape[0]).astype(np.int8)
file["spacing"] = np.asarray([spacing.z, spacing.y, spacing.x])

In [24]:
file

{'series_id': '1.3.6.1.4.1.55648.2.54139456488540773870566095168236824695',
 'images': array([[[-3024, -3024, -3024, ..., -3024, -3024, -3024],
         [-3024, -3024, -3024, ..., -3024, -3024, -3024],
         [-3024, -3024, -3024, ..., -3024, -3024, -3024],
         ...,
         [-3024, -3024, -3024, ..., -3024, -3024, -3024],
         [-3024, -3024, -3024, ..., -3024, -3024, -3024],
         [-3024, -3024, -3024, ..., -3024, -3024, -3024]],
 
        [[-3024, -3024, -3024, ..., -3024, -3024, -3024],
         [-3024, -3024, -3024, ..., -3024, -3024, -3024],
         [-3024, -3024, -3024, ..., -3024, -3024, -3024],
         ...,
         [-3024, -3024, -3024, ..., -3024, -3024, -3024],
         [-3024, -3024, -3024, ..., -3024, -3024, -3024],
         [-3024, -3024, -3024, ..., -3024, -3024, -3024]],
 
        [[-3024, -3024, -3024, ..., -3024, -3024, -3024],
         [-3024, -3024, -3024, ..., -3024, -3024, -3024],
         [-3024, -3024, -3024, ..., -3024, -3024, -3024],
         .

In [26]:
#| export 
def ctscan2dict(ctscan):
    file = {}
    spacing = ctscan.scan.spacing
    file["series_id"] = ctscan.series_instance_uid
    file["images"] = ctscan.scan.array.astype(np.int16)
    file["boxes"] =  np.empty((0, 6)) if len(ctscan.nodules) == 0 else np.asarray([i.gt.annot.bbox.xcyczcwhd for i in  ctscan.nodules])[:, [1, 0, 2, 4, 3, 5]]
    if np.any(file["boxes"]<0): breakpoint()
    file["labels"] = np.zeros(file["boxes"].shape[0]).astype(np.int8)
    file["spacing"] = np.asarray([spacing.z, spacing.y, spacing.x])
    return file

In [27]:
#| export 
def fixed_cache_transforms(lung_cache_dir: str = "", 
                           device: str="cuda:0",
                            roi: Optional[str] = None):
    from voxdet.utils import locate_cls
    from torchvision.transforms import Compose
    test_transforms = [
    dict(__class_fullname__ = "voxdet.tfsm.standard.StandardT",
         src_mode="yxzhwd", 
         img_src_mode="zyx"),
    dict(__class_fullname__="voxdet.tfsm.med.AddLungCache", 
         cache_dir=lung_cache_dir,
        device = device), 
    dict(__class_fullname__="voxdet.tfsm.med.CropLung",
         device = device,
        roi = roi),
    ]
    transforms = Compose([locate_cls(i) for i in test_transforms])
    return transforms 

In [28]:
transforms = fixed_cache_transforms(lung_cache_dir=lungmask_path,device="cuda:0")

In [29]:
transforms

Compose(
    voxdet.tfsm.standard.StandardT(src_mode='yxzhwd', img_src_mode='zyx')
    voxdet.tfsm.med.AddLungCache(cache_dir=Path('/cache/fast_data_nas8/qct/shubham/lung_mask_cache_det/recist_segmed'))
    voxdet.tfsm.med.CropLung(margin=(2, 5, 5))
)

In [30]:
#| export 
def generate_cache(meta_root, csv_loc, transforms, save_dir_root):
    df = pd.read_csv(csv_loc)
    df = df.rename(columns = {"annot_union" : "annot" , "meta_union": "meta"})
    df = df.sort_values(by='scan_name')
    df['counter'] = df.groupby('scan_name').cumcount() + 1
    df['annot_id'] = df['scan_name'] + "_" + df['counter'].astype(str)
    df.drop(columns=['counter'], inplace=True)
    
    series_ids = fc.L(df.scan_name.unique().tolist())

    ds = CTAnnotLoader(scans_root=meta_root, csv_loc=df, series_ids=series_ids)
    fc.Path(save_dir_root).mkdir(exist_ok=True)
    for n, sid in enumerate(series_ids):
        save_path = fc.Path(save_dir_root)/f"{sid}.safetensors"
        if save_path.exists(): 
            logger.info(f"{n}-{sid} already exists")
            continue
        try : 
            ctscan = ds[sid]
        except:
            logger.debug(f"scan is not present for this {sid}")
            continue
        for annot in ctscan.nodules:
            annot.gt.annot.bbox = annot.gt.annot.mask.to_boxc()
            logger.debug(annot.gt.annot.bbox)

        file = ctscan2dict(ctscan)

        rads = []
        for att in ctscan.nodules : 
            rads.append(df.loc[df["annot_id"] == att.gt.annot_id , "rads"].values[0])

        file["rads"] = np.asarray(rads)
        
        try:
            file2 = transforms(file)

            if np.any(file2["boxes"]<0): 
                logger.info(f"failed: {sid}")
                continue
            logger.info(f"{n}-{sid}-{file['boxes'].shape[0]}")

            file2["images"]=file2["images"].astype(np.int16)
            del file2["lung_mask"] #delete lung mask it is not required.
            del file2["series_id"] # delete series_id as we are saving the file with series_id.

            save_file(file2, save_path)
        
        except Exception as e :
            logger.error(e)
            continue

In [33]:
from tqdm.auto import tqdm

In [34]:
for n, sid in tqdm(enumerate(series_ids)):
    save_path = fc.Path(save_dir_root)/f"{sid}.safetensors"
    if save_path.exists(): 
        logger.info(f"{n}-{sid} already exists")
        continue
    try : 
        ctscan = ds[sid]
    except:
        logger.debug(f"scan is not present for this {sid}")
        continue
    for annot in ctscan.nodules:
        annot.gt.annot.bbox = annot.gt.annot.mask.to_boxc()
        logger.debug(annot.gt.annot.bbox)

    file = ctscan2dict(ctscan)
    
    rads = []
    for att in ctscan.nodules : 
        rads.append(df.loc[df["annot_id"] == att.gt.annot_id , "rads"].values[0])
        
    file["rads"] = np.asarray(rads)
        
    try:
        file2 = transforms(file)
        if np.any(file2["boxes"]<0): 
            logger.info(f"failed: {series_id}")
            continue
        #file["boxes"].shape == 0:
        logger.info(f"{n}-{sid}-{file['boxes'].shape[0]}")

        file2["images"]=file2["images"].astype(np.int16)
        del file2["lung_mask"] #delete lung mask it is not required.
        del file2["series_id"] # delete series_id as we are saving the file with series_id.

        save_file(file2, save_path)
        
    except Exception as e :
        print(e)
        continue
    except Exception as KeyboardInterrupt :
        break
    

0it [00:00, ?it/s]

2024-07-12 15:58:48.886 | DEBUG    | __main__:<module>:13 - xc=320.5 yc=315.5 w=48.0 h=86.0 zc=68.5 d=16.0
2024-07-12 15:58:49.021 | DEBUG    | __main__:<module>:13 - xc=215.5 yc=352.5 w=10.0 h=10.0 zc=56.0 d=3.0
2024-07-12 15:58:49.970 | INFO     | __main__:<module>:29 - 0-1.3.6.1.4.1.55648.2.100906588966924643291251806135982148156-2
2024-07-12 15:58:54.932 | DEBUG    | __main__:<module>:13 - xc=179.5 yc=274.5 w=8.0 h=10.0 zc=24.5 d=2.0
2024-07-12 15:58:55.040 | DEBUG    | __main__:<module>:13 - xc=324.5 yc=312.0 w=74.0 h=53.0 zc=78.5 d=20.0
2024-07-12 15:58:55.646 | INFO     | __main__:<module>:29 - 1-1.3.6.1.4.1.55648.2.10097783833891709104772397994918883764-2
2024-07-12 15:58:56.548 | DEBUG    | __main__:<module>:13 - xc=276.0 yc=262.5 w=147.0 h=144.0 zc=55.0 d=3.0
2024-07-12 15:58:56.670 | DEBUG    | __main__:<module>:13 - xc=156.5 yc=246.5 w=20.0 h=18.0 zc=19.5 d=4.0
2024-07-12 15:58:56.790 | DEBUG    | __main__:<module>:13 - xc=338.5 yc=320.5 w=18.0 h=18.0 zc=54.5 d=2.0
2024-07-

2024-07-12 15:59:26.326 | DEBUG    | __main__:<module>:13 - xc=122.0 yc=285.5 w=11.0 h=10.0 zc=49.5 d=2.0
2024-07-12 15:59:26.435 | DEBUG    | __main__:<module>:13 - xc=202.5 yc=152.5 w=18.0 h=18.0 zc=58.0 d=5.0
2024-07-12 15:59:26.541 | DEBUG    | __main__:<module>:13 - xc=319.5 yc=206.5 w=44.0 h=62.0 zc=42.0 d=17.0
2024-07-12 15:59:26.650 | DEBUG    | __main__:<module>:13 - xc=152.0 yc=254.5 w=11.0 h=14.0 zc=39.5 d=2.0
2024-07-12 15:59:27.699 | INFO     | __main__:<module>:29 - 11-1.3.6.1.4.1.55648.2.117920632119097483675993131555963786967-5
2024-07-12 15:59:31.487 | DEBUG    | __main__:<module>:13 - xc=390.0 yc=301.5 w=31.0 h=24.0 zc=31.5 d=2.0
2024-07-12 15:59:31.571 | DEBUG    | __main__:<module>:13 - xc=328.0 yc=321.5 w=31.0 h=44.0 zc=24.5 d=4.0
2024-07-12 15:59:31.655 | DEBUG    | __main__:<module>:13 - xc=320.0 yc=256.0 w=93.0 h=109.0 zc=40.5 d=6.0
2024-07-12 15:59:31.738 | DEBUG    | __main__:<module>:13 - xc=334.5 yc=330.0 w=22.0 h=21.0 zc=31.5 d=2.0
2024-07-12 15:59:32.227 |

2024-07-12 16:00:18.289 | INFO     | __main__:<module>:29 - 23-1.3.6.1.4.1.55648.2.134890697256781795948795173672589477251-1
2024-07-12 16:00:23.365 | DEBUG    | __main__:<module>:13 - xc=206.5 yc=326.5 w=18.0 h=20.0 zc=33.0 d=5.0
2024-07-12 16:00:23.476 | DEBUG    | __main__:<module>:13 - xc=302.0 yc=289.5 w=27.0 h=46.0 zc=38.0 d=15.0
2024-07-12 16:00:23.585 | DEBUG    | __main__:<module>:13 - xc=323.5 yc=333.5 w=52.0 h=20.0 zc=28.0 d=7.0
2024-07-12 16:00:24.509 | INFO     | __main__:<module>:29 - 24-1.3.6.1.4.1.55648.2.134994477946138374466181765108799468380-3
2024-07-12 16:00:25.311 | DEBUG    | __main__:<module>:13 - xc=132.0 yc=247.5 w=137.0 h=192.0 zc=47.5 d=44.0
2024-07-12 16:00:25.432 | DEBUG    | __main__:<module>:13 - xc=395.5 yc=224.5 w=42.0 h=46.0 zc=55.5 d=4.0
2024-07-12 16:00:25.553 | DEBUG    | __main__:<module>:13 - xc=176.0 yc=316.0 w=33.0 h=41.0 zc=57.0 d=7.0
2024-07-12 16:00:26.247 | INFO     | __main__:<module>:29 - 25-1.3.6.1.4.1.55648.2.135125859372039383256189910

2024-07-12 16:01:08.644 | INFO     | __main__:<module>:29 - 32-1.3.6.1.4.1.55648.2.143269084056311933401221834387335124246-2
2024-07-12 16:01:09.590 | DEBUG    | __main__:<module>:13 - xc=169.0 yc=315.5 w=95.0 h=90.0 zc=84.0 d=33.0
2024-07-12 16:01:10.655 | INFO     | __main__:<module>:29 - 33-1.3.6.1.4.1.55648.2.143576664367286034813779833250188041103-1
2024-07-12 16:01:15.494 | DEBUG    | __main__:<module>:13 - xc=182.5 yc=206.5 w=66.0 h=82.0 zc=54.0 d=9.0
2024-07-12 16:01:16.117 | INFO     | __main__:<module>:29 - 34-1.3.6.1.4.1.55648.2.145217178544939084717431758276170588109-1
2024-07-12 16:01:20.683 | DEBUG    | __main__:<module>:13 - xc=141.0 yc=306.5 w=119.0 h=78.0 zc=38.0 d=19.0
2024-07-12 16:01:21.289 | INFO     | __main__:<module>:29 - 35-1.3.6.1.4.1.55648.2.145330535270152884610271849421709398095-1
2024-07-12 16:01:22.100 | DEBUG    | __main__:<module>:13 - xc=285.0 yc=377.0 w=245.0 h=51.0 zc=66.0 d=1.0
2024-07-12 16:01:22.194 | DEBUG    | __main__:<module>:13 - xc=427.0 yc=

...

2024-07-12 16:48:44.474 | INFO     | __main__:<module>:29 - 539-1.3.6.1.4.1.55648.2.931147624288302458662048706808975556730-1
2024-07-12 16:48:46.528 | DEBUG    | __main__:<module>:13 - xc=327.5 yc=324.5 w=58.0 h=52.0 zc=43.5 d=8.0
2024-07-12 16:48:47.495 | INFO     | __main__:<module>:29 - 540-1.3.6.1.4.1.55648.2.931244035206111279411086813145354813150-1
2024-07-12 16:48:53.437 | DEBUG    | __main__:<module>:13 - xc=114.0 yc=286.0 w=7.0 h=7.0 zc=58.5 d=2.0
2024-07-12 16:48:53.535 | DEBUG    | __main__:<module>:13 - xc=160.5 yc=308.0 w=16.0 h=15.0 zc=22.5 d=2.0
2024-07-12 16:48:53.637 | DEBUG    | __main__:<module>:13 - xc=154.0 yc=320.0 w=141.0 h=109.0 zc=73.5 d=36.0
2024-07-12 16:48:54.417 | INFO     | __main__:<module>:29 - 541-1.3.6.1.4.1.55648.2.932822867901055242141410697514780569105-3
2024-07-12 16:48:57.998 | DEBUG    | __main__:<module>:13 - xc=196.0 yc=293.0 w=59.0 h=61.0 zc=45.5 d=4.0
2024-07-12 16:48:58.492 | INFO     | __main__:<module>:29 - 542-1.3.6.1.4.1.55648.2.9351002

2024-07-12 16:49:56.786 | DEBUG    | __main__:<module>:13 - xc=98.5 yc=194.0 w=6.0 h=7.0 zc=63.5 d=2.0
2024-07-12 16:49:56.870 | DEBUG    | __main__:<module>:13 - xc=198.5 yc=129.5 w=8.0 h=8.0 zc=58.0 d=1.0
2024-07-12 16:49:57.410 | INFO     | __main__:<module>:29 - 551-1.3.6.1.4.1.55648.2.951227066780224184006503387384614574501-6
2024-07-12 16:50:03.314 | DEBUG    | __main__:<module>:13 - xc=312.0 yc=281.0 w=19.0 h=17.0 zc=26.0 d=3.0
2024-07-12 16:50:03.418 | DEBUG    | __main__:<module>:13 - xc=370.0 yc=317.0 w=13.0 h=11.0 zc=39.5 d=2.0
2024-07-12 16:50:03.522 | DEBUG    | __main__:<module>:13 - xc=313.0 yc=280.5 w=23.0 h=22.0 zc=29.5 d=4.0
2024-07-12 16:50:04.535 | INFO     | __main__:<module>:29 - 552-1.3.6.1.4.1.55648.2.951349899544548320020499945618796321859-3
2024-07-12 16:50:09.764 | DEBUG    | __main__:<module>:13 - xc=300.5 yc=324.0 w=44.0 h=81.0 zc=43.5 d=12.0
2024-07-12 16:50:10.269 | INFO     | __main__:<module>:29 - 553-1.3.6.1.4.1.55648.2.95142802474590051712604568091082

2024-07-12 16:51:01.492 | INFO     | __main__:<module>:29 - 561-1.3.6.1.4.1.55648.2.96404103795747378592782278013116028178-1
2024-07-12 16:51:07.420 | DEBUG    | __main__:<module>:13 - xc=174.5 yc=208.5 w=42.0 h=76.0 zc=42.0 d=21.0
2024-07-12 16:51:08.727 | INFO     | __main__:<module>:29 - 562-1.3.6.1.4.1.55648.2.967814659063802850444302127195035134640-1
2024-07-12 16:51:19.300 | DEBUG    | __main__:<module>:13 - xc=336.5 yc=235.0 w=54.0 h=53.0 zc=43.5 d=6.0
2024-07-12 16:51:19.480 | DEBUG    | __main__:<module>:13 - xc=350.0 yc=239.0 w=11.0 h=11.0 zc=53.0 d=1.0
2024-07-12 16:51:21.367 | INFO     | __main__:<module>:29 - 563-1.3.6.1.4.1.55648.2.968508545224130848353296149157571285916-2
2024-07-12 16:51:27.967 | DEBUG    | __main__:<module>:13 - xc=154.0 yc=332.5 w=17.0 h=14.0 zc=102.0 d=3.0
2024-07-12 16:51:28.104 | DEBUG    | __main__:<module>:13 - xc=378.5 yc=201.5 w=132.0 h=166.0 zc=31.5 d=42.0
2024-07-12 16:51:29.796 | INFO     | __main__:<module>:29 - 564-1.3.6.1.4.1.55648.2.9685

2024-07-12 16:52:39.702 | DEBUG    | __main__:<module>:13 - xc=296.0 yc=217.0 w=45.0 h=39.0 zc=23.5 d=4.0
2024-07-12 16:52:39.773 | DEBUG    | __main__:<module>:13 - xc=315.0 yc=338.5 w=65.0 h=68.0 zc=34.0 d=13.0
2024-07-12 16:52:39.848 | DEBUG    | __main__:<module>:13 - xc=242.0 yc=211.0 w=329.0 h=207.0 zc=21.5 d=2.0
2024-07-12 16:52:40.353 | INFO     | __main__:<module>:29 - 575-1.3.6.1.4.1.55648.2.98317789153448279897029756106635302772-8
2024-07-12 16:52:46.794 | DEBUG    | __main__:<module>:13 - xc=211.0 yc=221.0 w=101.0 h=83.0 zc=89.5 d=36.0
2024-07-12 16:52:48.353 | INFO     | __main__:<module>:29 - 576-1.3.6.1.4.1.55648.2.983432984873138074603604693077566258112-1
2024-07-12 16:52:54.754 | DEBUG    | __main__:<module>:13 - xc=209.5 yc=325.0 w=30.0 h=33.0 zc=48.0 d=11.0
2024-07-12 16:52:54.881 | DEBUG    | __main__:<module>:13 - xc=316.5 yc=209.0 w=10.0 h=15.0 zc=50.5 d=4.0
2024-07-12 16:52:56.912 | INFO     | __main__:<module>:29 - 577-1.3.6.1.4.1.55648.2.98581658270322334804794

2024-07-12 16:53:49.514 | DEBUG    | __main__:<module>:13 - xc=398.5 yc=177.0 w=14.0 h=11.0 zc=83.0 d=3.0
2024-07-12 16:53:49.628 | DEBUG    | __main__:<module>:13 - xc=143.0 yc=320.0 w=11.0 h=7.0 zc=57.5 d=2.0
2024-07-12 16:53:49.744 | DEBUG    | __main__:<module>:13 - xc=186.0 yc=275.0 w=19.0 h=15.0 zc=57.0 d=3.0
2024-07-12 16:53:49.860 | DEBUG    | __main__:<module>:13 - xc=195.0 yc=254.0 w=79.0 h=149.0 zc=60.0 d=25.0
2024-07-12 16:53:49.975 | DEBUG    | __main__:<module>:13 - xc=167.5 yc=205.5 w=24.0 h=28.0 zc=61.5 d=4.0
2024-07-12 16:53:50.095 | DEBUG    | __main__:<module>:13 - xc=184.0 yc=257.5 w=15.0 h=12.0 zc=53.5 d=4.0
2024-07-12 16:53:51.055 | INFO     | __main__:<module>:29 - 586-1.3.6.1.4.1.55648.2.995815970422211566566332602993538546237-8
2024-07-12 16:53:54.861 | DEBUG    | __main__:<module>:13 - xc=169.5 yc=362.5 w=6.0 h=8.0 zc=98.0 d=1.0
2024-07-12 16:53:54.968 | DEBUG    | __main__:<module>:13 - xc=391.0 yc=380.0 w=7.0 h=7.0 zc=66.5 d=2.0
2024-07-12 16:53:55.079 | DEB

In [ ]:
import os
import shutil

def transfer_files(source_folder, destination_folder):
    # Ensure destination folder exists
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    # Iterate over all files in the source folder
    for filename in os.listdir(source_folder):
        source_file = os.path.join(source_folder, filename)
        destination_file = os.path.join(destination_folder, filename)
        
        # Check if the file already exists in the destination folder
        if not os.path.exists(destination_file):
            shutil.copy2(source_file, destination_file)
            print(f'Transferred: {filename}')
        else:
            print(f'Skipped (already exists): {filename}')

source_folder = '/cache/fast_data_nas8/qct/shubham/cache_3d/nlst/'
destination_folder = '/cache/shubham/cache_safe/nlst/'

transfer_files(source_folder, destination_folder)


In [29]:
#| hide
import nbdev; nbdev.nbdev_export()